In [1]:
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta, timezone
import requests
import pandas as pd
import json

load_dotenv()

API_KEY = os.getenv("RAPID_API_KEY")
API_HOST = os.getenv("API_HOST")

HEADERS = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": API_HOST
}

In [ ]:
def fetch_airport(iata):
    url = f"https://{API_HOST}/airports/iata/{iata}"
    r = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    return r.json()

In [ ]:
airport_data = fetch_airport("DEL")

In [ ]:
airport_data

In [2]:
import pytz

In [3]:
def fetch_flights(iata: str, date_str: str):
    """
    Fetch ALL flights for a full calendar day (IST)
    using two API calls:
      - 00:00 → 12:00
      - 12:01 → 23:59

    Returns a clean, packed structure:
    {
        "iata": "DEL",
        "date": "2024-12-14",
        "timezone": "Asia/Kolkata",
        "arrivals": [...],
        "departures": [...],
        "meta": {...}
    }
    """

    tz = pytz.timezone("Asia/Kolkata")

    day_start = tz.localize(datetime.strptime(date_str, "%Y-%m-%d"))
    mid_day = day_start + timedelta(hours=12)
    day_end = day_start + timedelta(hours=23, minutes=59)

    ranges = [
        (day_start, mid_day),
        (mid_day + timedelta(minutes=1), day_end)
    ]

    arrivals = []
    departures = []

    # Used for deduplication
    seen_arrivals = set()
    seen_departures = set()

    querystring = {
        "withLeg": "true",
        "direction": "Both",
        "withCancelled": "true",
        "withCodeshared": "true",
        "withCargo": "true",
        "withPrivate": "true",
        "withLocation": "true"
    }

    for start, end in ranges:
        from_ts = start.strftime("%Y-%m-%dT%H:%M")
        to_ts = end.strftime("%Y-%m-%dT%H:%M")

        url = f"https://{API_HOST}/flights/airports/iata/{iata}/{from_ts}/{to_ts}"

        print(f"➡️ Fetching flights: {from_ts} → {to_ts}")

        r = requests.get(url, headers=HEADERS, params=querystring)
        r.raise_for_status()

        data = r.json()

        # ---- Pack arrivals ----
        for flight in data.get("arrivals", []):
            fid = flight.get("flight", {}).get("iataNumber") or id(flight)
            if fid not in seen_arrivals:
                arrivals.append(flight)
                seen_arrivals.add(fid)

        # ---- Pack departures ----
        for flight in data.get("departures", []):
            fid = flight.get("flight", {}).get("iataNumber") or id(flight)
            if fid not in seen_departures:
                departures.append(flight)
                seen_departures.add(fid)

    return {
        "iata": iata,
        "date": date_str,
        "timezone": "Asia/Kolkata",
        "arrivals": arrivals,
        "departures": departures,
        "meta": {
            "arrival_count": len(arrivals),
            "departure_count": len(departures),
            "windows_used": len(ranges)
        }
    }

In [4]:
flights = fetch_flights("DEL", "2024-12-14")

➡️ Fetching flights: 2024-12-14T00:00 → 2024-12-14T12:00
➡️ Fetching flights: 2024-12-14T12:01 → 2024-12-14T23:59


In [5]:
departed_flights = pd.DataFrame(flights["departures"])

In [6]:
departed_flights

,departure,arrival,number,callSign,status,codeshareStatus,isCargo,aircraft,airline
0,"{'scheduledTime': {'utc': '2024-12-14 00:05Z',...","{'airport': {'icao': 'VAID', 'iata': 'IDR', 'n...",AI 2913,AIC2913,Departed,IsOperator,False,"{'modeS': '8014F3', 'model': 'Airbus A320 NEO'}","{'name': 'Air India', 'iata': 'AI', 'icao': 'A..."
1,"{'scheduledTime': {'utc': '2024-12-14 00:05Z',...","{'airport': {'icao': 'VEBS', 'iata': 'BBI', 'n...",IX 1151,NaN,Departed,IsOperator,False,{'model': 'Boeing 737'},"{'name': 'Air India Express', 'iata': 'IX', 'i..."
2,"{'scheduledTime': {'utc': '2024-12-14 00:05Z',...","{'airport': {'icao': 'CYVR', 'iata': 'YVR', 'n...",AI 185,AIC185,Departed,IsOperator,False,"{'reg': 'VT-ALX', 'modeS': '800C3E', 'model': ...","{'name': 'Air India', 'iata': 'AI', 'icao': 'A..."
3,"{'scheduledTime': {'utc': '2024-12-14 00:10Z',...","{'airport': {'icao': 'OBBI', 'iata': 'BAH', 'n...",GF 131,GFA131,Departed,IsOperator,False,"{'reg': 'A9C-NF', 'modeS': '8940C3', 'model': ...","{'name': 'Gulf Air', 'iata': 'GF', 'icao': 'GFA'}"
4,"{'scheduledTime': {'utc': '2024-12-14 00:15Z',...","{'airport': {'icao': 'VOTP', 'iata': 'TIR', 'n...",6E 286,IGO286,Departed,IsOperator,False,"{'reg': 'VT-IJY', 'modeS': '8013C9', 'model': ...","{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
...,...,...,...,...,...,...,...,...,...
604,"{'scheduledTime': {'utc': '2024-12-14 17:30Z',...","{'airport': {'icao': 'VABB', 'iata': 'BOM', 'n...",AI 2439,AIC2439,Departed,IsOperator,False,"{'reg': 'VT-TNP', 'modeS': '801396', 'model': ...","{'name': 'Air India', 'iata': 'AI', 'icao': 'A..."
605,"{'scheduledTime': {'utc': '2024-12-14 18:25Z',...","{'airport': {'icao': 'VABB', 'iata': 'BOM', 'n...",6E 519,NaN,Departed,Unknown,False,{'model': 'Airbus A320 NEO'},"{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
606,"{'scheduledTime': {'utc': '2024-12-14 18:25Z',...","{'airport': {'icao': 'KEWR', 'iata': 'EWR', 'n...",UA 83,UAL83,Departed,IsOperator,False,"{'reg': 'N26970', 'modeS': 'A2A3D2', 'model': ...","{'name': 'United', 'iata': 'UA', 'icao': 'UAL'}"
607,"{'scheduledTime': {'utc': '2024-12-14 18:25Z',...","{'airport': {'icao': 'VECC', 'iata': 'CCU', 'n...",SG 712,NaN,Departed,Unknown,False,{'model': 'Boeing 737'},"{'name': 'SpiceJet', 'iata': 'SG', 'icao': 'SEJ'}"


In [ ]:
flights["departures"][0]

In [7]:
arrival_flights = pd.DataFrame(flights["arrivals"])

In [11]:
arrival_flights

,departure,arrival,number,callSign,status,codeshareStatus,isCargo,aircraft,airline
0,"{'airport': {'name': 'Almaty'}, 'quality': []}","{'scheduledTime': {'utc': '2024-12-14 00:05Z',...",6E 1802,IGO1802,Arrived,IsOperator,False,"{'reg': 'VT-IUD', 'modeS': '800D58', 'model': ...","{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
1,"{'airport': {'name': 'Singapore'}, 'quality': []}","{'scheduledTime': {'utc': '2024-12-14 00:10Z',...",SQ 402,SIA402,Arrived,IsOperator,False,"{'reg': '9V-SCB', 'modeS': '76CC62', 'model': ...","{'name': 'Singapore', 'iata': 'SQ', 'icao': 'S..."
2,"{'airport': {'name': 'Kuwait'}, 'quality': []}","{'scheduledTime': {'utc': '2024-12-14 00:30Z',...",KU 383,NaN,Arrived,Unknown,False,{'model': 'Airbus A320 NEO'},"{'name': 'Kuwait', 'iata': 'KU', 'icao': 'KAC'}"
3,"{'airport': {'name': 'Tashkent'}, 'quality': []}","{'scheduledTime': {'utc': '2024-12-14 00:15Z',...",6E 1806,NaN,Arrived,Unknown,False,{'model': 'Airbus A320 NEO'},"{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
4,"{'airport': {'name': 'Jomo Kenyatta'}, 'qualit...","{'scheduledTime': {'utc': '2024-12-14 00:30Z',...",AI 962,AIC962,Arrived,IsOperator,False,"{'reg': 'VT-ANA', 'modeS': '8005EB', 'model': ...","{'name': 'Air India', 'iata': 'AI', 'icao': 'A..."
...,...,...,...,...,...,...,...,...,...
602,"{'airport': {'icao': 'VOVZ', 'iata': 'VTZ', 'n...","{'scheduledTime': {'utc': '2024-12-14 17:30Z',...",6E 2776,NaN,Arrived,Unknown,False,{'model': 'Airbus A320'},"{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
603,"{'airport': {'name': 'GOA (MOPA)'}, 'quality':...","{'scheduledTime': {'utc': '2024-12-14 18:40Z',...",AI 862,NaN,Arrived,Unknown,False,{'model': 'Airbus A320 NEO'},"{'name': 'Air India', 'iata': 'AI', 'icao': 'A..."
604,"{'airport': {'icao': 'VILK', 'iata': 'LKO', 'n...","{'scheduledTime': {'utc': '2024-12-14 18:55Z',...",6E 2058,NaN,Expected,Unknown,False,{'model': 'Airbus A320'},"{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"
605,"{'airport': {'icao': 'VABB', 'iata': 'BOM', 'n...","{'scheduledTime': {'utc': '2024-12-14 19:00Z',...",6E 651,IGO651,Expected,IsOperator,False,"{'modeS': '80173E', 'model': 'Airbus A320'}","{'name': 'IndiGo', 'iata': '6E', 'icao': 'IGO'}"


In [ ]:
flights["arrivals"][0]

In [15]:
def flights_to_dataframe(flights: dict, iata: str) -> pd.DataFrame:
    rows = []

    # -------- DEPARTURES --------
    for f in flights.get("departures", []):
        airline_code = f.get("airline", {}).get("iata")
        flight_number = f.get("number")

        scheduled_dep = f.get("departure", {}).get("scheduledTime", {}).get("utc")
        actual_dep = f.get("departure", {}).get("revisedTime", {}).get("utc") or scheduled_dep

        scheduled_arr = f.get("arrival", {}).get("scheduledTime", {}).get("utc")
        actual_arr = f.get("arrival", {}).get("revisedTime", {}).get("utc") or scheduled_arr

        origin = f.get("departure", {}).get("airport", {}).get("iata") or iata
        destination = f.get("arrival", {}).get("airport", {}).get("iata") or f.get("airline", {}).get("iata")

        flight_id = f"{airline_code}_{flight_number}_{scheduled_dep}"

        rows.append({
            "flight_id": flight_id,
            "flight_number": flight_number,
            "aircraft_registration": f.get("aircraft", {}).get("reg"),
            "origin_iata": origin,
            "destination_iata": destination,
            "scheduled_departure": scheduled_dep,
            "actual_departure": actual_dep,
            "scheduled_arrival": scheduled_arr,
            "actual_arrival": actual_arr,
            "status": f.get("status"),
            "airline_code": airline_code
        })

    # -------- ARRIVALS --------
    for f in flights.get("arrivals", []):
        airline_code = f.get("airline", {}).get("iata")
        flight_number = f.get("number")

        scheduled_arr = f.get("arrival", {}).get("scheduledTime", {}).get("utc")
        actual_arr = f.get("arrival", {}).get("revisedTime", {}).get("utc") or scheduled_arr

        scheduled_dep = f.get("departure", {}).get("scheduledTime", {}).get("utc")
        actual_dep = f.get("departure", {}).get("revisedTime", {}).get("utc") or scheduled_dep

        origin = f.get("departure", {}).get("airport", {}).get("iata") or f.get("airline", {}).get("iata")
        destination = f.get("arrival", {}).get("airport", {}).get("iata") or iata

        flight_id = f"{airline_code}_{flight_number}_{scheduled_arr}"

        rows.append({
            "flight_id": flight_id,
            "flight_number": flight_number,
            "aircraft_registration": f.get("aircraft", {}).get("reg"),
            "origin_iata": origin,
            "destination_iata": destination,
            "scheduled_departure": scheduled_dep,
            "actual_departure": actual_dep,
            "scheduled_arrival": scheduled_arr,
            "actual_arrival": actual_arr,
            "status": f.get("status"),
            "airline_code": airline_code
        })

    df = pd.DataFrame(rows)

    # Remove duplicates (same flight seen in arrival + departure)
    df.drop_duplicates(subset=["flight_id"], inplace=True)

    return df

flights_to_dataframe(flights, "DEL")


,flight_id,flight_number,aircraft_registration,origin_iata,destination_iata,scheduled_departure,actual_departure,scheduled_arrival,actual_arrival,status,airline_code
0,AI_AI 2913_2024-12-14 00:05Z,AI 2913,None,DEL,IDR,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 01:50Z,2024-12-14 01:50Z,Departed,AI
1,IX_IX 1151_2024-12-14 00:05Z,IX 1151,None,DEL,BBI,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 02:15Z,2024-12-14 02:15Z,Departed,IX
2,AI_AI 185_2024-12-14 00:05Z,AI 185,VT-ALX,DEL,YVR,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 14:30Z,2024-12-14 14:30Z,Departed,AI
3,GF_GF 131_2024-12-14 00:10Z,GF 131,A9C-NF,DEL,BAH,2024-12-14 00:10Z,2024-12-14 00:10Z,2024-12-14 05:00Z,2024-12-14 05:00Z,Departed,GF
4,6E_6E 286_2024-12-14 00:15Z,6E 286,VT-IJY,DEL,TIR,2024-12-14 00:15Z,2024-12-14 00:15Z,2024-12-14 02:55Z,2024-12-14 02:55Z,Departed,6E
...,...,...,...,...,...,...,...,...,...,...,...
1211,6E_6E 2776_2024-12-14 17:30Z,6E 2776,None,VTZ,DEL,2024-12-14 15:00Z,2024-12-14 15:00Z,2024-12-14 17:30Z,2024-12-14 18:17Z,Arrived,6E
1212,AI_AI 862_2024-12-14 18:40Z,AI 862,None,AI,DEL,None,None,2024-12-14 18:40Z,2024-12-14 18:17Z,Arrived,AI
1213,6E_6E 2058_2024-12-14 18:55Z,6E 2058,None,LKO,DEL,2024-12-14 17:40Z,2024-12-14 17:40Z,2024-12-14 18:55Z,2024-12-14 18:18Z,Expected,6E
1214,6E_6E 651_2024-12-14 19:00Z,6E 651,None,BOM,DEL,2024-12-14 16:45Z,2024-12-14 16:45Z,2024-12-14 19:00Z,2024-12-14 18:19Z,Expected,6E


In [16]:
df_flight = flights_to_dataframe(flights, "DEL")

In [17]:
df_flight

,flight_id,flight_number,aircraft_registration,origin_iata,destination_iata,scheduled_departure,actual_departure,scheduled_arrival,actual_arrival,status,airline_code
0,AI_AI 2913_2024-12-14 00:05Z,AI 2913,None,DEL,IDR,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 01:50Z,2024-12-14 01:50Z,Departed,AI
1,IX_IX 1151_2024-12-14 00:05Z,IX 1151,None,DEL,BBI,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 02:15Z,2024-12-14 02:15Z,Departed,IX
2,AI_AI 185_2024-12-14 00:05Z,AI 185,VT-ALX,DEL,YVR,2024-12-14 00:05Z,2024-12-14 00:05Z,2024-12-14 14:30Z,2024-12-14 14:30Z,Departed,AI
3,GF_GF 131_2024-12-14 00:10Z,GF 131,A9C-NF,DEL,BAH,2024-12-14 00:10Z,2024-12-14 00:10Z,2024-12-14 05:00Z,2024-12-14 05:00Z,Departed,GF
4,6E_6E 286_2024-12-14 00:15Z,6E 286,VT-IJY,DEL,TIR,2024-12-14 00:15Z,2024-12-14 00:15Z,2024-12-14 02:55Z,2024-12-14 02:55Z,Departed,6E
...,...,...,...,...,...,...,...,...,...,...,...
1211,6E_6E 2776_2024-12-14 17:30Z,6E 2776,None,VTZ,DEL,2024-12-14 15:00Z,2024-12-14 15:00Z,2024-12-14 17:30Z,2024-12-14 18:17Z,Arrived,6E
1212,AI_AI 862_2024-12-14 18:40Z,AI 862,None,AI,DEL,None,None,2024-12-14 18:40Z,2024-12-14 18:17Z,Arrived,AI
1213,6E_6E 2058_2024-12-14 18:55Z,6E 2058,None,LKO,DEL,2024-12-14 17:40Z,2024-12-14 17:40Z,2024-12-14 18:55Z,2024-12-14 18:18Z,Expected,6E
1214,6E_6E 651_2024-12-14 19:00Z,6E 651,None,BOM,DEL,2024-12-14 16:45Z,2024-12-14 16:45Z,2024-12-14 19:00Z,2024-12-14 18:19Z,Expected,6E


In [24]:
import time

In [25]:
# -------------------------------
# HELPERS
# -------------------------------

def parse_delay_minutes(delay_str):
    """
    Converts 'HH:MM:SS' or '-HH:MM:SS' → minutes (int)
    """
    if not delay_str:
        return 0

    sign = -1 if delay_str.startswith("-") else 1
    h, m, _ = delay_str.replace("-", "").split(":")
    return sign * (int(h) * 60 + int(m))


# -------------------------------
# FETCH DELAY WINDOWS (SAFE)
# -------------------------------

def fetch_airport_delay_windows(iata: str, date_str: str):
    tz = pytz.timezone("Asia/Kolkata")

    day_start = tz.localize(datetime.strptime(date_str, "%Y-%m-%d"))
    day_end = day_start + timedelta(days=1)

    # IMPORTANT: delays API returns PREVIOUS 2 HOURS
    current = day_start + timedelta(hours=2)

    windows = []

    while current <= day_end:
        ts = current.strftime("%Y-%m-%dT%H:%M")
        url = f"https://{API_HOST}/airports/iata/{iata}/delays/{ts}"

        print(f"➡️ Fetching delay window ending at {ts}")

        r = requests.get(url, headers=HEADERS)

        # RapidAPI can silently throttle with empty body
        if r.status_code != 200 or not r.text.strip():
            print(f"⚠️ Skipped {ts} | status={r.status_code}")
            current += timedelta(hours=2)
            time.sleep(2)
            continue

        try:
            data = r.json()
        except ValueError:
            print(f"❌ Non-JSON response at {ts}")
            current += timedelta(hours=2)
            time.sleep(2)
            continue

        windows.append(data)

        current += timedelta(hours=2)
        time.sleep(2)  # REQUIRED to avoid throttling

    return windows


# -------------------------------
# AGGREGATE DAILY DELAYS
# -------------------------------

def aggregate_daily_delays(iata: str, date_str: str, windows: list):
    """
    IMPORTANT:
    Delays API returns SNAPSHOTS → use MAX, not SUM
    """

    max_total = 0
    max_delayed = 0
    max_cancelled = 0
    median_delays = []

    for w in windows:
        dep = w.get("departuresDelayInformation", {})
        arr = w.get("arrivalsDelayInformation", {})

        total = dep.get("numTotal", 0) + arr.get("numTotal", 0)
        delayed = dep.get("numQualifiedTotal", 0) + arr.get("numQualifiedTotal", 0)
        cancelled = dep.get("numCancelled", 0) + arr.get("numCancelled", 0)

        max_total = max(max_total, total)
        max_delayed = max(max_delayed, delayed)
        max_cancelled = max(max_cancelled, cancelled)

        median_delays.extend([
            parse_delay_minutes(dep.get("medianDelay")),
            parse_delay_minutes(arr.get("medianDelay"))
        ])

    median_delays = [d for d in median_delays if d != 0]

    avg_delay_min = (
        sum(median_delays) // len(median_delays)
        if median_delays else 0
    )

    median_delay_min = (
        sorted(median_delays)[len(median_delays)//2]
        if median_delays else 0
    )

    return {
        "airport_iata": iata,
        "delay_date": date_str,
        "total_flights": max_total,
        "delayed_flights": max_delayed,
        "avg_delay_min": avg_delay_min,
        "median_delay_min": median_delay_min,
        "canceled_flights": max_cancelled
    }


# -------------------------------
# RUN ETL
# -------------------------------

def run_airport_delay_etl(iata: str, date_str: str):
    print(f"\n===== AIRPORT DELAY ETL | {iata} | {date_str} =====")

    windows = fetch_airport_delay_windows(iata, date_str)

    if not windows:
        print("❌ No delay windows fetched")
        return None

    daily_delay = aggregate_daily_delays(iata, date_str, windows)

    print("\n✅ DAILY DELAY SUMMARY")
    for k, v in daily_delay.items():
        print(f"{k}: {v}")

    return daily_delay


# -------------------------------
# EXECUTE (EXAMPLE)
# -------------------------------

daily_delay = run_airport_delay_etl("DEL", "2024-12-14")


===== AIRPORT DELAY ETL | DEL | 2024-12-14 =====
➡️ Fetching delay window ending at 2024-12-14T02:00
⚠️ Skipped 2024-12-14T02:00 | status=204
➡️ Fetching delay window ending at 2024-12-14T04:00
⚠️ Skipped 2024-12-14T04:00 | status=204
➡️ Fetching delay window ending at 2024-12-14T06:00
➡️ Fetching delay window ending at 2024-12-14T08:00
➡️ Fetching delay window ending at 2024-12-14T10:00
➡️ Fetching delay window ending at 2024-12-14T12:00
➡️ Fetching delay window ending at 2024-12-14T14:00
➡️ Fetching delay window ending at 2024-12-14T16:00
➡️ Fetching delay window ending at 2024-12-14T18:00
➡️ Fetching delay window ending at 2024-12-14T20:00
➡️ Fetching delay window ending at 2024-12-14T22:00
➡️ Fetching delay window ending at 2024-12-15T00:00

✅ DAILY DELAY SUMMARY
airport_iata: DEL
delay_date: 2024-12-14
total_flights: 141
delayed_flights: 141
avg_delay_min: -2
median_delay_min: -6
canceled_flights: 4


In [27]:
windows = fetch_airport_delay_windows("DEL", "2024-12-14")

windows
# if not windows:
#     print("❌ No delay windows fetched")
#     return None

# daily_delay = aggregate_daily_delays(iata, date_str, windows)

# print("\n✅ DAILY DELAY SUMMARY")
# for k, v in daily_delay.items():
#     print(f"{k}: {v}")


➡️ Fetching delay window ending at 2024-12-14T02:00
⚠️ Skipped 2024-12-14T02:00 | status=204
➡️ Fetching delay window ending at 2024-12-14T04:00
⚠️ Skipped 2024-12-14T04:00 | status=204
➡️ Fetching delay window ending at 2024-12-14T06:00
➡️ Fetching delay window ending at 2024-12-14T08:00
➡️ Fetching delay window ending at 2024-12-14T10:00
➡️ Fetching delay window ending at 2024-12-14T12:00
➡️ Fetching delay window ending at 2024-12-14T14:00
➡️ Fetching delay window ending at 2024-12-14T16:00
➡️ Fetching delay window ending at 2024-12-14T18:00
➡️ Fetching delay window ending at 2024-12-14T20:00
➡️ Fetching delay window ending at 2024-12-14T22:00
➡️ Fetching delay window ending at 2024-12-15T00:00


[{'airportIcao': 'VIDP',
  'from': {'utc': '2024-12-13 22:30Z', 'local': '2024-12-14 04:00+05:30'},
  'to': {'utc': '2024-12-14 00:30Z', 'local': '2024-12-14 06:00+05:30'},
  'departuresDelayInformation': {'numTotal': 48,
   'numQualifiedTotal': 48,
   'numCancelled': 2,
   'medianDelay': '00:00:00',
   'delayIndex': 0.0},
  'arrivalsDelayInformation': {'numTotal': 21,
   'numQualifiedTotal': 21,
   'numCancelled': 0,
   'medianDelay': '-00:07:00',
   'delayIndex': 0.0}},
 {'airportIcao': 'VIDP',
  'from': {'utc': '2024-12-14 00:30Z', 'local': '2024-12-14 06:00+05:30'},
  'to': {'utc': '2024-12-14 02:30Z', 'local': '2024-12-14 08:00+05:30'},
  'departuresDelayInformation': {'numTotal': 72,
   'numQualifiedTotal': 72,
   'numCancelled': 3,
   'medianDelay': '00:00:00',
   'delayIndex': 0.0},
  'arrivalsDelayInformation': {'numTotal': 34,
   'numQualifiedTotal': 34,
   'numCancelled': 1,
   'medianDelay': '-00:12:00',
   'delayIndex': 0.0}},
 {'airportIcao': 'VIDP',
  'from': {'utc': '20

In [32]:
windows[0]

{'airportIcao': 'VIDP',
 'from': {'utc': '2024-12-13 22:30Z', 'local': '2024-12-14 04:00+05:30'},
 'to': {'utc': '2024-12-14 00:30Z', 'local': '2024-12-14 06:00+05:30'},
 'departuresDelayInformation': {'numTotal': 48,
  'numQualifiedTotal': 48,
  'numCancelled': 2,
  'medianDelay': '00:00:00',
  'delayIndex': 0.0},
 'arrivalsDelayInformation': {'numTotal': 21,
  'numQualifiedTotal': 21,
  'numCancelled': 0,
  'medianDelay': '-00:07:00',
  'delayIndex': 0.0}}

In [38]:
def windows_to_dataframe(windows: list) -> pd.DataFrame:
    rows = []

    for w in windows:
        row = {
            "airport_icao": w.get("airportIcao"),

            # ---- FROM / TO (split) ----
            "from_utc": w.get("from", {}).get("utc"),
            "from_local": w.get("from", {}).get("local"),
            "to_utc": w.get("to", {}).get("utc"),
            "to_local": w.get("to", {}).get("local"),

            # ---- DEPARTURES ----
            "dep_total": w.get("departuresDelayInformation", {}).get("numTotal"),
            "dep_delayed": w.get("departuresDelayInformation", {}).get("numTotal") - w.get("departuresDelayInformation", {}).get("numQualifiedTotal"),
            "dep_cancelled": w.get("departuresDelayInformation", {}).get("numCancelled"),
            "dep_median_delay": w.get("departuresDelayInformation", {}).get("medianDelay"),
            "dep_delay_index": w.get("departuresDelayInformation", {}).get("delayIndex"),

            # ---- ARRIVALS ----
            "arr_total": w.get("arrivalsDelayInformation", {}).get("numTotal"),
            "arr_delayed": w.get("arrivalsDelayInformation", {}).get("numTotal") - w.get("arrivalsDelayInformation", {}).get("numQualifiedTotal"),
            "arr_cancelled": w.get("arrivalsDelayInformation", {}).get("numCancelled"),
            "arr_median_delay": w.get("arrivalsDelayInformation", {}).get("medianDelay"),
            "arr_delay_index": w.get("arrivalsDelayInformation", {}).get("delayIndex"),
        }

        rows.append(row)

    df = pd.DataFrame(rows)

    # Optional: parse datetime columns
    for col in ["from_utc", "from_local", "to_utc", "to_local"]:
        df[col] = pd.to_datetime(df[col], errors="coerce")

    return df

windows_df = windows_to_dataframe(windows)

In [39]:
windows_df

,airport_icao,from_utc,from_local,to_utc,to_local,dep_total,dep_delayed,dep_cancelled,dep_median_delay,dep_delay_index,arr_total,arr_delayed,arr_cancelled,arr_median_delay,arr_delay_index
0,VIDP,2024-12-13 22:30:00+00:00,2024-12-14 04:00:00+05:30,2024-12-14 00:30:00+00:00,2024-12-14 06:00:00+05:30,48,0,2,00:00:00,0.00,21,0,0,-00:07:00,0.0
1,VIDP,2024-12-14 00:30:00+00:00,2024-12-14 06:00:00+05:30,2024-12-14 02:30:00+00:00,2024-12-14 08:00:00+05:30,72,0,3,00:00:00,0.00,34,0,1,-00:12:00,0.0
2,VIDP,2024-12-14 02:30:00+00:00,2024-12-14 08:00:00+05:30,2024-12-14 04:30:00+00:00,2024-12-14 10:00:00+05:30,49,0,0,00:00:00,0.00,70,0,0,-00:06:00,0.0
3,VIDP,2024-12-14 04:30:00+00:00,2024-12-14 10:00:00+05:30,2024-12-14 06:30:00+00:00,2024-12-14 12:00:00+05:30,81,0,1,00:00:00,0.00,60,0,0,-00:07:00,0.0
4,VIDP,2024-12-14 06:30:00+00:00,2024-12-14 12:00:00+05:30,2024-12-14 08:30:00+00:00,2024-12-14 14:00:00+05:30,62,0,1,00:00:00,0.00,54,0,0,-00:02:00,0.0
5,VIDP,2024-12-14 08:30:00+00:00,2024-12-14 14:00:00+05:30,2024-12-14 10:30:00+00:00,2024-12-14 16:00:00+05:30,66,0,2,00:00:00,0.00,74,0,1,00:00:00,0.0
6,VIDP,2024-12-14 10:30:00+00:00,2024-12-14 16:00:00+05:30,2024-12-14 12:30:00+00:00,2024-12-14 18:00:00+05:30,61,0,0,00:00:00,0.00,78,0,0,-00:03:00,0.0
7,VIDP,2024-12-14 12:30:00+00:00,2024-12-14 18:00:00+05:30,2024-12-14 14:30:00+00:00,2024-12-14 20:00:00+05:30,66,0,2,00:00:00,0.00,64,0,0,-00:06:00,0.0
8,VIDP,2024-12-14 14:30:00+00:00,2024-12-14 20:00:00+05:30,2024-12-14 16:30:00+00:00,2024-12-14 22:00:00+05:30,64,0,1,00:00:00,0.00,67,0,0,-00:05:00,0.0
9,VIDP,2024-12-14 16:30:00+00:00,2024-12-14 22:00:00+05:30,2024-12-14 18:30:00+00:00,2024-12-15 00:00:00+05:30,38,0,2,00:32:00,1.89,69,0,0,00:00:00,0.0


In [57]:
filtered_df = df_flight[
    (df_flight["origin_iata"].str.upper() == "DEL") &
    (
        (
            df_flight["scheduled_departure"].notna() &
            df_flight["actual_departure"].notna()
        )
    )
]


In [58]:
filtered_df["scheduled_departure"] = pd.to_datetime(
    filtered_df["scheduled_departure"], errors="coerce"
)
filtered_df["actual_departure"] = pd.to_datetime(
    filtered_df["actual_departure"], errors="coerce"
)


filtered_df["departure_delay_min"] = (
    filtered_df["actual_departure"] -
    filtered_df["scheduled_departure"]
).dt.total_seconds() / 60

filtered_df["departure_delay_min"] = filtered_df["departure_delay_min"].clip(lower=0)



C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\4142628310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["scheduled_departure"] = pd.to_datetime(
C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\4142628310.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["actual_departure"] = pd.to_datetime(
C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\4142628310.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [55]:
filtered_df = df_flight[
    (df_flight["destination_iata"].str.upper() == "DEL") &
    (
        (
            df_flight["scheduled_arrival"].notna() &
            df_flight["actual_arrival"].notna()
        )
    )
]


filtered_df["scheduled_arrival"] = pd.to_datetime(
    filtered_df["scheduled_arrival"], errors="coerce"
)
filtered_df["actual_arrival"] = pd.to_datetime(
    filtered_df["actual_arrival"], errors="coerce"
)


filtered_df["arrival_delay_min"] = (
    filtered_df["actual_arrival"] -
    filtered_df["scheduled_arrival"]
).dt.total_seconds() / 60

filtered_df["arrival_delay_min"] = filtered_df["arrival_delay_min"].clip(lower=0)




C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\3382221107.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["scheduled_arrival"] = pd.to_datetime(
C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\3382221107.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["actual_arrival"] = pd.to_datetime(
C:\Users\Praveen Meganathan\AppData\Local\Temp\ipykernel_392\3382221107.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [59]:
filtered_df

,flight_id,flight_number,aircraft_registration,origin_iata,destination_iata,scheduled_departure,actual_departure,scheduled_arrival,actual_arrival,status,airline_code,departure_delay_min
0,AI_AI 2913_2024-12-14 00:05Z,AI 2913,None,DEL,IDR,2024-12-14 00:05:00+00:00,2024-12-14 00:05:00+00:00,2024-12-14 01:50Z,2024-12-14 01:50Z,Departed,AI,0.0
1,IX_IX 1151_2024-12-14 00:05Z,IX 1151,None,DEL,BBI,2024-12-14 00:05:00+00:00,2024-12-14 00:05:00+00:00,2024-12-14 02:15Z,2024-12-14 02:15Z,Departed,IX,0.0
2,AI_AI 185_2024-12-14 00:05Z,AI 185,VT-ALX,DEL,YVR,2024-12-14 00:05:00+00:00,2024-12-14 00:05:00+00:00,2024-12-14 14:30Z,2024-12-14 14:30Z,Departed,AI,0.0
3,GF_GF 131_2024-12-14 00:10Z,GF 131,A9C-NF,DEL,BAH,2024-12-14 00:10:00+00:00,2024-12-14 00:10:00+00:00,2024-12-14 05:00Z,2024-12-14 05:00Z,Departed,GF,0.0
4,6E_6E 286_2024-12-14 00:15Z,6E 286,VT-IJY,DEL,TIR,2024-12-14 00:15:00+00:00,2024-12-14 00:15:00+00:00,2024-12-14 02:55Z,2024-12-14 02:55Z,Departed,6E,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
604,AI_AI 2439_2024-12-14 17:30Z,AI 2439,VT-TNP,DEL,BOM,2024-12-14 17:30:00+00:00,2024-12-14 18:21:00+00:00,2024-12-14 19:55Z,2024-12-14 19:55Z,Departed,AI,51.0
605,6E_6E 519_2024-12-14 18:25Z,6E 519,None,DEL,BOM,2024-12-14 18:25:00+00:00,2024-12-14 18:24:00+00:00,2024-12-14 20:45Z,2024-12-14 20:45Z,Departed,6E,0.0
606,UA_UA 83_2024-12-14 18:25Z,UA 83,N26970,DEL,EWR,2024-12-14 18:25:00+00:00,2024-12-14 18:25:00+00:00,2024-12-15 11:25Z,2024-12-15 11:15Z,Departed,UA,0.0
607,SG_SG 712_2024-12-14 18:25Z,SG 712,None,DEL,CCU,2024-12-14 18:25:00+00:00,2024-12-14 18:25:00+00:00,2024-12-14 20:35Z,2024-12-14 20:35Z,Departed,SG,0.0


In [60]:
from db_connection import get_connection

In [61]:
url = "https://aerodatabox.p.rapidapi.com/aircrafts/reg/VT-ALX"

querystring = {"withRegistrations":"true","withImage":"true"}

headers = {
	"x-rapidapi-key": "c327ce5b76msh4057af5cb8bb06ep1b5294jsn8c8e086ee032",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'id': 14497, 'reg': 'VT-ALX', 'active': True, 'serial': '36322', 'hexIcao': '800C3E', 'airlineName': 'Air India', 'iataCodeShort': '777', 'icaoCode': 'B777', 'model': 'B773', 'modelCode': 'B777-337ER', 'numSeats': 342, 'rolloutDate': '2018-02-02', 'firstFlightDate': '2018-02-02', 'deliveryDate': '2018-03-09', 'registrationDate': '2018-03-09', 'typeName': 'Boeing 777', 'numEngines': 2, 'engineType': 'Jet', 'isFreighter': False, 'productionLine': 'Boeing 777', 'ageYears': 7.9, 'verified': True, 'image': {'url': 'https://farm66.staticflickr.com/65535/48188306701_2605a1e0c1_z.jpg', 'webUrl': 'https://www.flickr.com/photos/58415659@N00/48188306701/', 'author': 'kitmasterbloke', 'title': 'VT-ALJ - Boeing 777-337(ER) - Air India LHR 280619', 'description': 'London Heathrow Terminal 4 and taxying 28 June 2019, Flight KL1021 AMS-LHR', 'license': 'AttributionCC', 'htmlAttributions': ['Original of "<span property=\'dc:title\' itemprop=\'name\'>VT-ALJ - Boeing 777-337(ER) - Air India LHR 280619</